In [1]:
def save_image(block):
    import cartopy.crs as ccrs
    import matplotlib.pyplot as plt
    if sum(block.shape) > 0:
        # workaround 1:
        # xarray passes a zero shaped array to infer what this function returns. 
        # we can't run plot, so avoid doing that
        f = plt.figure()
        ax = f.subplots(1, 1, subplot_kw={"projection": ccrs.PlateCarree()}, squeeze=True)

        # xarray plotting goodness is available here!
        block.plot(ax=ax, robust=True, vmin=5, vmax=28, cmap=mpl.cm.Spectral_r, cbar_kwargs={"extend": "both"})
   
        # on pangeo.io, this will need some tweaking to work with gcsfs.
        # haven't tried that. On cheyenne, it works beautifully.
        f.savefig(f"images/aqua/{block.time.values[0]}.png", dpi=180)
        plt.close(f)

    # workaround 2:
    # map_blocks expects to receive an xarray thing back.
    # Just send back one value. If we send back "block" that's like computing the whole dataset!
    return block["time"]

In [2]:
import numpy as np 
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from datetime import datetime
import dask.datasets as dd
import dask

import niskine
import mplniw.plot as pl
import gvpy as gv
import mplniw.sigp as sp
import mplniw.utils as ut


In [3]:
conf = niskine.io.load_config()
lon, lat, bottom_depth = niskine.io.mooring_location(mooring=1)
root_dir = '/Users/zcasparcohen/codes/NIW/'
figdir = '/Users/zcasparcohen/Documents/Figures/'

In [4]:
mooring=1
wind_work = xr.open_dataset(root_dir+'data/wind_work/ni_wind_work_niskine_m1.nc').load()
gridded = xr.open_dataset(root_dir+'data/gridded/M%s_gridded_simple_merge_gaps_filled.nc'%mooring).load()


In [5]:
ds_filtered = sp.ni_bandpass_adcp(gridded)
ds_filt,wkb = sp.calc_ni_eke(ds_filtered)
ds_filt

<xarray.Dataset>
Dimensions:       (z: 106, time: 72966, adcp: 3)
Coordinates:
  * time          (time) datetime64[ns] 2019-05-17T16:00:00 ... 2020-10-05T08...
  * z             (z) int64 0 16 32 48 64 80 ... 1600 1616 1632 1648 1664 1680
  * adcp          (adcp) int64 3109 9408 13481
Data variables:
    u             (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    v             (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    xducer_depth  (adcp, time) float64 nan 230.8 233.2 ... 817.6 817.3 817.0
    temperature   (adcp, time) float64 nan 7.478 7.453 ... 5.513 5.512 5.512
    bpu           (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    bpv           (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    n2            (z, time) float64 2.251e-06 2.251e-06 ... 1.244e-06 1.244e-06
    N             (z, time) float64 0.0015 0.0015 0.0015 ... 0.001115 0.001115
    ni_eke        (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    project:       NISKINe
    mooring:       M1
    lon:           -21.198829621395166
    lat:           59.101445324829214
    bottom_depth:  2881.0

In [6]:
#MLD loading
mld_all = xr.open_dataset(conf.data.input.argo_mld)
mld_da = xr.DataArray(data=mld_all.mld_da_mean.data, coords=dict(lon=mld_all.lon.data, lat=mld_all.lat.data, month=mld_all.month.data), dims=('lat', 'lon', 'month'), name='da')
mld_dt = xr.DataArray(data=mld_all.mld_dt_mean.data, coords=dict(lon=mld_all.lon.data, lat=mld_all.lat.data, month=mld_all.month.data), dims=('lat', 'lon', 'month'), name='dt')


In [7]:
#Local MLD
mld = mld_da.sel(lon=lon,lat=lat,method='nearest')
mld_m1 = xr.open_dataarray(conf.data.ml.mld)
mld_m1_argo = xr.open_dataset(conf.data.ml.mld_argo).interp(lon=lon,lat=lat)

In [8]:
# Defining events ("a la main"):
ti = ['2019-11-09T02:00:00.000000000','2020-01-03T01:00:00.000000000',
      '2020-02-13T02:00:00.000000000','2020-03-06T12:00:00.000000000']
tf = ['2019-11-22T09:00:00.000000000','2020-01-19T14:00:00.000000000',
      '2020-02-26T19:00:00.000000000','2020-03-29T04:00:00.000000000']
months=[11,1,2,3]

In [9]:
ds_filt['ke'] = 0.5*(ds_filt.u**2+ds_filt.v**2)
ds_filt['eke'] = 0.5*((ds_filt.u-ds_filt.u.mean())**2+(ds_filt.v-ds_filt.v.mean())**2)
ds_filt['bpu_wkb'] = ds_filt['bpu']*wkb
ds_filt['bpv_wkb'] = ds_filt['bpv']*wkb
ds_filt.ni_eke.attrs.update({'long_name':'Kinetic energy', 'units':r'$m^2.s^{-2}$'})
ds_filt.bpu_wkb.attrs.update({'long_name':'Zonal velocity', 'units':r'$m.s^{-1}$'})
ds_filt.bpv_wkb.attrs.update({'long_name':'Meridional velocity', 'units':r'$m.s^{-1}$'})

In [11]:
ds_filt

<xarray.Dataset>
Dimensions:       (z: 106, time: 72966, adcp: 3)
Coordinates:
  * time          (time) datetime64[ns] 2019-05-17T16:00:00 ... 2020-10-05T08...
  * z             (z) int64 0 16 32 48 64 80 ... 1600 1616 1632 1648 1664 1680
  * adcp          (adcp) int64 3109 9408 13481
Data variables: (12/13)
    u             (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    v             (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    xducer_depth  (adcp, time) float64 nan 230.8 233.2 ... 817.6 817.3 817.0
    temperature   (adcp, time) float64 nan 7.478 7.453 ... 5.513 5.512 5.512
    bpu           (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    bpv           (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    ...            ...
    N             (z, time) float64 0.0015 0.0015 0.0015 ... 0.001115 0.001115
    ni_eke        (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    ke            (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    eke           (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    bpu_wkb       (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
    bpv_wkb       (z, time) float64 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    project:       NISKINe
    mooring:       M1
    lon:           -21.198829621395166
    lat:           59.101445324829214
    bottom_depth:  2881.0

In [13]:

#for i,t in enumerate(_time[int(8323*6)::6]):
def plot(ds,time,i):
    fig = plt.figure(tight_layout=True,figsize=(5,8))
    gs = gridspec.GridSpec(3,2)
    ax0 = fig.add_subplot(gs[2,:])
    ax0.axvline(time.values,c='k')
    wind_work['NISKINE M1 mode1 wind-work'].sel(time=_time,method='nearest').plot(ax=ax0,zorder=-1);
    ax0.grid();ax0.set_title('');ax0.set_xlabel('time');ax0.set_ylabel('Wind work')
    #ax0.tick_params(bottom=False,labelbottom=False)
    #fig,ax = plt.subplots(1,2,figsize=(10,5))
    for j,v in enumerate(['bpu_wkb','bpv_wkb']):
        ax = fig.add_subplot(gs[:2,j])
        ds.sel(time=time,z=slice(100,None))[v].plot(ax=ax,y='z',yincrease=False,zorder=1,
                                                    xlim=(ds.bpu_wkb.min(),ds.bpu_wkb.max()),ylim=(1500,0))
    #ds_filt.sel(time=t,z=slice(100,None)).bpv_wkb.plot(ax=ax[1],y='z',yincrease=False,zorder=1);
        #ax.axhline(mld_m1_argo.da_m1.sel(month=months[index]),ls='--',lw=1.5,c='k',zorder=-1)
        ax.axhline(mld_m1.sel(time=time, method='nearest'),ls='--',lw=1.5,c='k',zorder=-1)
        #.plot(ax=ax,x='time',c='k',zorder=-1) 
        ax.grid();ax.set_title('');ax.set_xlabel('');ax.set_title(v[2]+r'$_{wkb}$ [m/s]')
    ax.tick_params(left=False,labelleft=False);ax.set_ylabel('');
    fig.suptitle(datetime.utcfromtimestamp(time.values.astype(int) * 1e-9).strftime('%m/%d/%Y, %H:%M:%S'),x=0.6)
    #plt.show();
    fig.savefig(root_dir+'_Figures/event%s_time%s.png'%(index,i));
    #fig.clear()
    plt.close(fig)

In [14]:
index=0
_time = ds_filt.sel(time=slice(ti[index],tf[index])).time

In [15]:
@dask.delayed

tasks = [plot(ds_filt, time, i) for i,time in enumerate(_time.values)]

# just do first few 
#dask.compute(tasks[:5], scheduler='processes', num_workers=4)

SyntaxError: invalid syntax (1203505440.py, line 3)

In [21]:
client.close()

Remove client Client-4232c238-4600-11ee-9bd2-821e962368e0
Received 'close-stream' from tcp://100.90.197.130:52014; closing.
Remove client Client-4232c238-4600-11ee-9bd2-821e962368e0
Close client connection: Client-4232c238-4600-11ee-9bd2-821e962368e0
